# Iris Dataset Example - Logistic Regressor

In [4]:
import sys

!{sys.executable} -m pip install skl2onnx==1.9.0
!{sys.executable} -m pip install onnx==1.10.1
!{sys.executable} -m pip install onnxruntime==1.8.1

     |████████████████████████████████| 239 kB 30.4 MB/s            
  Attempting uninstall: skl2onnx
    Found existing installation: skl2onnx 1.12
    Uninstalling skl2onnx-1.12:
      Successfully uninstalled skl2onnx-1.12
     |████████████████████████████████| 12.3 MB 18.9 MB/s            
  Attempting uninstall: onnx
    Found existing installation: onnx 1.12.0
    Uninstalling onnx-1.12.0:
      Successfully uninstalled onnx-1.12.0
     |████████████████████████████████| 4.5 MB 26.5 MB/s            
  Attempting uninstall: onnxruntime
    Found existing installation: onnxruntime 1.10.0
    Uninstalling onnxruntime-1.10.0:
      Successfully uninstalled onnxruntime-1.10.0


In [5]:
from skl2onnx import to_onnx
from skl2onnx.common.data_types import FloatTensorType
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import onnxruntime as rt
import numpy as np

# create a pipeline object
pipe = make_pipeline(
    StandardScaler(),
    LogisticRegression()
)

# load the iris dataset and split it into train and test sets
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# fit the whole pipeline
pipe.fit(X_train, y_train)

# we can now use it like any other estimator
print(accuracy_score(pipe.predict(X_test), y_test))

onxx_pipeline = to_onnx(pipe, initial_types=[('sepalLength', FloatTensorType([None, 1])),
                                             ('sepalWidth', FloatTensorType([None, 1])),
                                             ('petalLength', FloatTensorType([None, 1])),
                                             ('petalWidth', FloatTensorType([None, 1]))])

with open("iris_lr.onnx", "wb") as f:
    f.write(onxx_pipeline.SerializeToString())

0.9736842105263158


In [6]:
sess = rt.InferenceSession("iris_lr.onnx")
inputs = {'sepalLength': [[5.0]], 'sepalWidth': [[3.5]], 'petalLength': [[4.0]], 'petalWidth': [[5.0]]}

pred_onx = sess.run(None, inputs)

In [7]:
print("prediction", pred_onx[0])
print("probabilites", pred_onx[1])

prediction [2]
probabilites [{0: 1.3849917479547003e-07, 1: 6.7487194428395014e-06, 2: 0.999993085861206}]
